# Tier 1: Arbitrage Bounds

**Learning Objectives:**
- Understand no-arbitrage price constraints
- Use bounds to catch pricing bugs
- Implement validation gates

**Key Constraints [T1]:**
- Call: 0 ≤ C ≤ S
- Put: 0 ≤ P ≤ Ke^(-rT)

**Duration:** ~15 minutes

In [ ]:
import numpy as np
from annuity_pricing.options.pricing.black_scholes import (
    black_scholes_call,
    black_scholes_put,
)

## 1. The Bounds

These bounds are **arbitrage constraints**, not model assumptions:

| Option | Lower Bound | Upper Bound | Why |
|--------|-------------|-------------|-----|
| Call | max(0, S·e^(-qT) - K·e^(-rT)) | S | Can't pay more than stock |
| Put | max(0, K·e^(-rT) - S·e^(-qT)) | K·e^(-rT) | Can't pay more than strike |

In [ ]:
def check_bounds(S, K, r, q, T, call_price, put_price):
    """Check if option prices satisfy arbitrage bounds."""
    forward_spot = S * np.exp(-q * T)
    disc_strike = K * np.exp(-r * T)
    
    # Call bounds
    call_lower = max(0, forward_spot - disc_strike)
    call_upper = S
    call_ok = call_lower <= call_price <= call_upper
    
    # Put bounds
    put_lower = max(0, disc_strike - forward_spot)
    put_upper = disc_strike
    put_ok = put_lower <= put_price <= put_upper
    
    return {
        'call_lower': call_lower, 'call_upper': call_upper, 'call_ok': call_ok,
        'put_lower': put_lower, 'put_upper': put_upper, 'put_ok': put_ok
    }

In [ ]:
# Normal case
S, K, r, q, sigma, T = 100, 100, 0.05, 0.02, 0.20, 1.0

call = black_scholes_call(S, K, r, q, sigma, T)
put = black_scholes_put(S, K, r, q, sigma, T)
bounds = check_bounds(S, K, r, q, T, call, put)

print(f"Prices: Call = ${call:.4f}, Put = ${put:.4f}")
print(f"Call bounds: [{bounds['call_lower']:.4f}, {bounds['call_upper']:.4f}]")
print(f"Put bounds:  [{bounds['put_lower']:.4f}, {bounds['put_upper']:.4f}]")
print(f"Status: Call {'✓' if bounds['call_ok'] else '✗'}, Put {'✓' if bounds['put_ok'] else '✗'}")

## 2. What Causes Bound Violations?

Most common cause: **Wrong time units**

In [ ]:
from scipy.stats import norm

def bs_call_time_bug(S, K, r, q, sigma, T_days):
    """BUG: T is in days but formula expects years."""
    T = T_days  # Should be T_days / 365!
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * np.exp(-q*T) * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

# Test with T in days (wrong!)
T_days = 365
call_bug = bs_call_time_bug(S, K, r, q, sigma, T_days)

print(f"Bug: T = {T_days} (days, should be years)")
print(f"Call price = ${call_bug:.4f}")
print(f"Spot price = ${S:.2f}")
print()
if call_bug > S:
    print(f"ARBITRAGE VIOLATION: Call (${call_bug:.2f}) > Spot (${S:.2f})!")
    print("This is impossible - you could sell the call and buy the stock for risk-free profit.")

## 3. Implement as Validation Gate

In [ ]:
def validate_option_price(option_type, price, S, K, r, q, T):
    """Raise error if price violates arbitrage bounds."""
    forward_spot = S * np.exp(-q * T)
    disc_strike = K * np.exp(-r * T)
    
    if option_type == 'call':
        lower = max(0, forward_spot - disc_strike)
        upper = S
    else:  # put
        lower = max(0, disc_strike - forward_spot)
        upper = disc_strike
    
    if price < lower or price > upper:
        raise ValueError(
            f"ARBITRAGE VIOLATION: {option_type} price {price:.4f} "
            f"outside bounds [{lower:.4f}, {upper:.4f}]"
        )
    return True

# Test validation
try:
    validate_option_price('call', call_bug, S, K, r, q, T_days)
except ValueError as e:
    print(f"Caught error: {e}")

## 4. Key Takeaways

1. **Call ≤ Spot** is the most common violation to check
2. **Time unit errors** are the usual cause
3. **Add bounds checks** to your pricing functions

## Next

Continue to **Tier 2** for product-specific deep dives.